To ensure correct text generation,  
we use **only one tokenizer (`tok_fast`)** — the same one used during training.

This prevents vocabulary mismatch and encoding errors (such as “Ð…”).  
The model is loaded from checkpoints and its embeddings are resized  
to match the tokenizer’s vocabulary.


In [ ]:
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast

# tokenizer
TOK_JSON = '/content/tokenizer_bpe/bpe_16000/tokenizer.json'
tok_fast = PreTrainedTokenizerFast(
    tokenizer_file=TOK_JSON,
    bos_token="<bos>", eos_token="<eos>", unk_token="<unk>", pad_token="<pad>"
)

# load model
mdl = GPT2LMHeadModel.from_pretrained(str(CKPT_DIR)).to(device)
mdl.eval()

if tok_fast.pad_token is None:
    tok_fast.add_special_tokens({"pad_token": "<pad>"})
if tok_fast.eos_token is None:
    tok_fast.add_special_tokens({"eos_token": "<eos>"})

mdl.resize_token_embeddings(len(tok_fast))
mdl.config.pad_token_id = tok_fast.pad_token_id
mdl.config.eos_token_id = tok_fast.eos_token_id

#little test
probe = "Күңелле һәм бәхетле көн"
ids = tok_fast(probe)["input_ids"]
print("RT:", tok_fast.decode(ids, skip_special_tokens=True))

# band number token list
vocab = tok_fast.get_vocab()
digit_ids = [[tid] for tok, tid in vocab.items() if any(ch.isdigit() for ch in tok)]

# generate func
def generate(prompt: str, max_new_tokens=80):
    enc = tok_fast(prompt, return_tensors="pt")
    with torch.no_grad():
        out = mdl.generate(
            input_ids=enc.input_ids.to(device),
            attention_mask=enc.attention_mask.to(device),
            max_new_tokens=max_new_tokens,
            num_beams=4,
            early_stopping=True,
            no_repeat_ngram_size=3,
            repetition_penalty=1.2,
            length_penalty=0.9,
            bad_words_ids=digit_ids or None,
            pad_token_id=tok_fast.pad_token_id,
            eos_token_id=tok_fast.eos_token_id,
        )
    text = tok_fast.decode(out[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text



RT: Күңелле һәм бәхетле көн


The model was tested on several Tatar-language questions.
It produced coherent and contextually relevant answers — although it did not respond directly to the questions but rather in a news-like manner — showing that it learned to "speak" Tatar after being trained from scratch.

# QA Demonstration

In [ ]:
p = [
    "Рәүф хәзрәт Гайнетдин нәрсә эшли?"]#What does Rauf Khazrat Gainetdin do?

print("Q:", p)
print("A:", generate(p, max_new_tokens=60))
print("------")


Q: ['Рәүф хәзрәт Гайнетдин нәрсә эшли?']
A: Рәүф хәзрәт Гайнетдин нәрсә эшли?“Ачык ишекләр көне” кысаларында “Иман” балалар-спорт комплексында, шулай ук “Зөя Казань” халыкара аэропорты хезмәткәрләре өчен дә оештырылган иде.
------


Translate:

Q: 'What does Rauf Khazrat Gainetdin do?'

A: What does Rauf Khazrat Gainetdin do? He organize the framework of the 'Open Doors Day' at the 'Iman' children's sports complex, as well as for the staff of 'Zoya Kazan' international airport.

In [ ]:
p = [

    "ТР Дәүләт Советы Рәисе турында кыскача яз.",  # Write briefly about the President of the State Council of the Republic of Tatarstan.
]
print("Q:", p)
print("A:", generate(p, max_new_tokens=60))
print("------")


Q: ['ТР Дәүләт Советы Рәисе турында кыскача яз.']
A: ТР Дәүләт Советы Рәисе турында кыскача яз.Матбугат конференциясендә ТР Премьер-министры Рөстәм Миңнеханов, ТР Президенты Минтимер Шәймиев һәм ТР Дәүләт Советы депутаты Марат Сафиуллин катнашты.
------


Translate:

Q: 'Write briefly about the President of the State Council of the Republic of Tatarstan.'

A: Write briefly about the President of the State Council of the Republic of Tatarstan. The press conference was attended by the Prime Minister of the Republic of Tatarstan Rustam Minnikhanov, the President of the Republic of Tatarstan Mintimer Shaimiev, and the Deputy of the State Council of the Republic of Tatarstan Marat Safiullin.

# Mini-benchmark

I also implemented a mini-benchmark with four different formulations of the question “Who is the mayor of Kazan?” — the model gives the same answer regardless of how the question is phrased.

In [ ]:
prompts = [
    "Казан шәһәре мэры кем?",  # Who is the mayor of Kazan?
    "Казан шәһәре мэры турында кыскача яз.",  # Briefly introduce the mayor of Kazan
    "Казан шәһәре мэры — ул кем?",  # Mayor of Kazan-who is he?
    "Казан шәһәре мэры турында өч җөмлә яз.",  # Introduce the mayor of Kazan in three sentences.
]

for p in prompts:
    print("Q:", p)
    print("A:", generate(p, max_new_tokens=30))
    print("------")


Q: Казан шәһәре мэры кем?
A: Казан шәһәре мэры кем?Бу хакта “Татар-информ” МА хәбәрчесенә Казан шәһәре мэры Илсур Метшин хәбәр итә.
------
Q: Казан шәһәре мэры турында кыскача яз.
A: Казан шәһәре мэры турында кыскача яз.Бу хакта “Татар-информ” МА хәбәрчесенә Казан шәһәре мэры Илсур Метшин хәбәр итә.
------
Q: Казан шәһәре мэры — ул кем?
A: Казан шәһәре мэры — ул кем?Бу хакта “Татар-информ” МА хәбәрчесенә Казан шәһәре мэры Илсур Метшин хәбәр итә.
------
Q: Казан шәһәре мэры турында өч җөмлә яз.
A: Казан шәһәре мэры турында өч җөмлә яз.Бу хакта “Татар-информ” МА хәбәрчесенә Казан шәһәре мэры Илсур Метшин хәбәр итә.
------


I also implemented two automatic evaluation metrics: notEcho and kwCover.
| Metric      | Meaning                                                          |
| ----------- | ---------------------------------------------------------------- |
| **notEcho** | Proportion of answers that do not simply repeat the question |
| **kwCover** | Coverage of key informational words (e.g., “Илсур Метшин”)       |


In [ ]:
# Kazan Mayor Mini-Benchmark
import re, torch, numpy as np

torch.manual_seed(42)




def not_echo(q, a):
    return q.strip().lower() not in a.strip().lower()

def has_keywords(a, kws):
    a_low = a.lower()
    return sum(1 for k in kws if k in a_low) / max(1, len(kws))

facts_kws = ["илсур", "метшин", "казан", "мэр"]

answers=["Казан шәһәре мэры кем?Бу хакта “Татар-информ” МА хәбәрчесенә Казан шәһәре мэры Илсур Метшин хәбәр итә."
"Казан шәһәре мэры турында кыскача яз.Бу хакта “Татар-информ” МА хәбәрчесенә Казан шәһәре мэры Илсур Метшин хәбәр итә."
"Казан шәһәре мэры — ул кем?Бу хакта “Татар-информ” МА хәбәрчесенә Казан шәһәре мэры Илсур Метшин хәбәр итә."
"Казан шәһәре мэры турында өч җөмлә яз.Бу хакта “Татар-информ” МА хәбәрчесенә Казан шәһәре мэры Илсур Метшин хәбәр итә."]
scores = {
    "notEcho": np.mean([not_echo(q, a) for q, a in zip(prompts, answers)]),
    "kwCover": np.mean([has_keywords(a, facts_kws) for a in answers]),
}

print("\n=== 📊 Summary (mini-benchmark) ===")
print({k: round(v, 2) for k, v in scores.items()})



=== 📊 Summary (mini-benchmark) ===
{'notEcho': 0.0, 'kwCover': 1.0}


Although the model tends to repeat the beginning of the question (notEcho = 0),
it achieves 100% coverage of factual keywords.

In [ ]:

def eval_tok_path(tok_path):
    tok_tmp = PreTrainedTokenizerFast(tokenizer_file=tok_path, bos_token="<bos>", eos_token="<eos>", unk_token="<unk>", pad_token="<pad>")
    sample = sample_lines(CLEAN_FILE, 1000)
    lens=[]
    for s in sample:
        enc = tok_tmp(s, return_tensors=None)
        lens.append(len(enc["input_ids"]))
    return np.mean(lens)

rows=[]
for vs, p in TOK_RUNS.items():
    rows.append((vs, round(eval_tok_path(p),2)))
rows = sorted(rows)
print("Vocab | avg_tokens_per_sentence")
for r in rows: print(r)


Vocab | avg_tokens_per_sentence
(8000, 27.37)
(16000, 24.3)
(32000, 22.22)


# Ablation Experiment (max_new_tokens)

I tested max_new_tokens = 40 / 80 / 160.
Short outputs (40) were more focused, while longer ones (80–160) resembled news-style paragraphs, often repeating the prompt or drifting off-topic.

All of these variants produced worse factual and contextual accuracy than the default parameters.

In [ ]:
# ===== Ablation experiment:
for mnt in [40, 80, 160]:
    print(f"\n=== max_new_tokens={mnt} ===")
    print("Q: Казан шәһәре мэры турында кыскача яз.")
    print("A:", generate("Казан шәһәре мэры турында кыскача яз.", max_new_tokens=mnt))



=== max_new_tokens=40 ===
Q: Казан шәһәре мэры турында кыскача яз.
A: Казан шәһәре мэры турында кыскача яз.Аның сүзләренә караганда, Татарстан Республикасында “ Сәламәтлек ” илкүләм проекты кысаларында республиканың барлык предприятиеләре һәм авыл хуҗалыгы продукциясе җитештерү буенча федераль үзәкләштерелгән.

=== max_new_tokens=80 ===
Q: Казан шәһәре мэры турында кыскача яз.
A: Казан шәһәре мэры турында кыскача яз.Чараны ТР Президенты Минтимер Шәймиев, “ Россия гражданнарына – яңа технологияләр ” проектын гамәлгә ашыру кысаларында Казан муниципаль берәмлеге башкарма комитеты рәисе урынбасары Равил Зарипов, Татарстан Республикасы Президенты Рөстәм Миңнеханов тапшырды.

=== max_new_tokens=160 ===
Q: Казан шәһәре мэры турында кыскача яз.
A: Казан шәһәре мэры турында кыскача яз.Кичә “Татар-информ” мәгълүмат агентлыгы белән берлектә “Ак Барс” “Бала”ка килде.


Translate:

=== max_new_tokens=40 ===

Q: Write briefly about the Mayor of Kazan.

A: Write briefly about the Mayor of Kazan. According to him, under the national project "Health" in the Republic of Tatarstan, all enterprises and agricultural production in the republic are centrally managed by the federal government.

=== max_new_tokens=80 ===

Q: Write briefly about the Mayor of Kazan.

A: Write briefly about the Mayor of Kazan. The event was presented by Tatarstan President Mintimer Shaimiev and, within the framework of the project "New Technologies for Russian Citizens," by the Deputy Chairman of the Executive Committee of the Kazan Municipality, Ravil Zaripov, and Tatarstan President Rustam Minnikhanov.

=== max_new_tokens=160 ===

Q: Write briefly about the Mayor of Kazan.

A: Write briefly about the Mayor of Kazan. Yesterday, together with the "Tatar-inform" news agency, he visited "Ak Bars" to come to "Bala".


**Conclusion:**

The model can sustain long text generation, indicating good contextual continuity,
but the default configuration still yields the most accurate and relevant results.

# Decoding Comparison (Beam Search vs Top-p Sampling)

| Strategy           | Expected Characteristics            | Actual Observation (Based on Outputs)                                                                                                                             |
| ------------------ | ----------------------------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Beam Search**    | Deterministic, syntactically stable | Tends to **repeat the prompt** and output generic, incomplete sentences without key facts (e.g., *“Казан шәһәре мэры турында кыскача яз. ... процентка күбрәк.”*) |
| **Top-p Sampling** | More diverse and creative           | Produces **news-like, verbose sentences** containing irrelevant entities and drifting away from the topic (e.g., *“... Ульяновски ... иҗтимагый оешмалар ...”*)   |


In [ ]:
import re, torch

def generate_beam(prompt, max_new_tokens=60):
    """Beam Search 生成"""
    enc = tok_fast(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        out = mdl.generate(
            **enc,
            max_new_tokens=max_new_tokens,
            num_beams=4,           # Beam Search
            early_stopping=True,
            no_repeat_ngram_size=3,
            repetition_penalty=1.2,
            pad_token_id=tok_fast.pad_token_id,
            eos_token_id=tok_fast.eos_token_id,
        )
    text = tok_fast.decode(out[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text


def generate_top_p(prompt, max_new_tokens=60, temperature=0.9, top_p=0.9):
    """Top-p / Nucleus Sampling 生成"""
    enc = tok_fast(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        out = mdl.generate(
            **enc,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            top_k=50,
            no_repeat_ngram_size=3,
            pad_token_id=tok_fast.pad_token_id,
            eos_token_id=tok_fast.eos_token_id,
        )
    text = tok_fast.decode(out[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text


#
prompt = "Казан шәһәре мэры турында кыскача яз."

beam_ans = generate_beam(prompt)
topp_ans = generate_top_p(prompt)

print("\n[Beam Search]\n", beam_ans)
print("\n[Top-p Sampling]\n", topp_ans)



[Beam Search]
 Казан шәһәре мэры турында кыскача яз.Аның сүзләренә караганда, узган елның шул чоры белән чагыштырганда, , процентка күбрәк.

[Top-p Sampling]
 Казан шәһәре мэры турында кыскача яз.Ульяновски өлкәсенең “Хәрәкәт” җәмгыяте җитәкчесе һәм Казан шәһәре башкарма комитетының яшьләр-районнарыннан, иҗтимагый оешмаларның җитәкчеләре, республиканың башка муниципаль берәмлекләр башлыклары да булды.


Translate:

[Beam Search]Write a brief note about the Mayor of Kazan. According to his words, compared to the same period last year, it increased by a certain percent.

[Top-p Sampling]Write a brief note about the Mayor of Kazan. The head of the “Movement” society in Ulyanovsk region, as well as representatives from youth districts of the Kazan city executive committee, leaders of public organizations, and heads of other municipal entities of the republic, were also present.

**Conclusion**:

Both decoding strategies performed worse than the default setup:

- Beam Search gave short but uninformative answers, repeating the question.

- Top-p Sampling generated longer yet off-topic text.
Thus, keeping the default decoding parameters remains the most reliable option.